In [ ]:
#importing all the dependencies
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import numpy as np
import random as rnd
rnd.seed(0)
import pandas as pd


In [ ]:
#Global Variables
DIR = os.getcwd()
#DIR is current directory
CATEGORY = ["buoy","cruise_ship","ferry_boat","freight_boat","gondola",
            "inflatable_boat","kayak","paper_boat","sailboat"]
IMG_SIZE=40
NUM_CLASSES = 9
TEST_DIR=os.path.join(DIR,"TEST")


In [ ]:
#helper functions
def one_hot(num):
    vec = np.zeros((9,1))
    vec[num]=1
    return vec

In [ ]:
#create training data
container=[]
for type in CATEGORY:
    path=os.path.join(DIR,type)
    class_num = CATEGORY.index(type)
    class_one_hot = one_hot(class_num)
    for img in os.listdir(path):
        try:
            img_arr=cv2.imread(os.path.join(path,img))
            new_arr = cv2.resize(img_arr,(IMG_SIZE,IMG_SIZE))
            new_arr=new_arr/255.00000
            new_arr = new_arr.reshape(1,IMG_SIZE,IMG_SIZE,3)
            container.append((new_arr,class_one_hot))
        except Exception as e:
            print("Incompatible File")
            pass
rnd.shuffle(container)
print("Train data scanned successfully")
print("# of images are")
print(len(container))
x_train = np.zeros((len(container),IMG_SIZE,IMG_SIZE,3))
y_train = np.zeros((len(container),NUM_CLASSES))
for i in range(len(container)):
    x_train[i] = container[i][0]
    y_train[i] = container[i][1].reshape(NUM_CLASSES,)
print("The shape of x_train , y_train")
print(x_train.shape)
print(y_train.shape)

In [ ]:
#have validation data




container_cv=[]
for type in CATEGORY:
    path=os.path.join(TEST_DIR,type)
    print(path)
    class_num = CATEGORY.index(type)
    class_one_hot = one_hot(class_num)
    for img in os.listdir(path):
        try:
            img_arr=cv2.imread(os.path.join(path,img))
            new_arr = cv2.resize(img_arr,(IMG_SIZE,IMG_SIZE))
            new_arr=new_arr/255
            new_arr = new_arr.reshape(1,IMG_SIZE,IMG_SIZE,3)
            container_cv.append((new_arr,class_one_hot))
        except Exception as e:
            print("Incompatible File")
            pass
rnd.shuffle(container_cv)
print("Cross Valiadtion data scanned successfully")
print("# of images are")
print(len(container))
x_test = np.zeros((len(container_cv),IMG_SIZE,IMG_SIZE,3))
y_test = np.zeros((len(container_cv),NUM_CLASSES))
for i in range(len(container_cv)):
    x_test[i] = container_cv[i][0]
    y_test[i] = container_cv[i][1].reshape(NUM_CLASSES,)
print("The shape of x_cross_validation , y_cross_validation")
print(x_test.shape)
print(y_test.shape)

In [ ]:
#create the model

model=tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,input_shape=(IMG_SIZE,IMG_SIZE,3),
                          kernel_size=(5, 5), strides=(1, 1),
                            activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    tf.keras.layers.Conv2D(64,kernel_size=(3, 3), strides=(1, 1),
                            activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(NUM_CLASSES,activation='softmax')
])
print(model.summary())
model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='categorical_crossentropy',metrics=['accuracy'])
history=model.fit(x_train,y_train,
                  epochs=200,
                 validation_data=(x_test,y_test))
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

